# Research Agentic Workflow

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
import time
import json
from IPython.display import Markdown, display
from langchain_community.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from collections import defaultdict
from langgraph.graph import StateGraph
from IPython.display import Image, display
from langgraph.graph import START, END
from typing_extensions import TypedDict, NotRequired
from typing import Dict
from prompts import reflector_prompt, json_transformer_prompt, reform_researcher_prompt


## Components

In [3]:
# Quesion Reflector
reflector_llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)

reflector_output_parser = StrOutputParser()

reflector_chain = reflector_prompt | reflector_llm | reflector_output_parser

In [4]:
# JSON Transformer
json_transformer_llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    model_kwargs={
        "stream": False,
        "response_format": {"type": "json_object"},
    }
)

json_transformer_output_parser = JsonOutputParser()

json_transformer_chain = json_transformer_prompt | json_transformer_llm | json_transformer_output_parser

In [5]:
# tool = DuckDuckGoSearchResults(num_results=10, safesearch="off")
tool = DuckDuckGoSearchRun(num_results=10, safesearch="off")

In [6]:
def reform_research(question, results):
    reform_researcher_llm = ChatGroq(model="llama-3.2-90b-text-preview", temperature=1)
    # reform_researcher_llm = ChatOllama(model="llama3.2", temperature=1)
    
    reform_researcher_output_parser = StrOutputParser()

    chain = reform_researcher_prompt | reform_researcher_llm | reform_researcher_output_parser

    response = chain.invoke({"question": question, "results": results})
    
    return response

In [7]:
def search_insights(questions: dict):
    question_no = 0
    research_result = "---"
    for key, question in questions.items():
        question_no+=1
        if question_no == 3: break
        # time.sleep(10)
        results = tool.invoke(question)
        reformed_results = reform_research(question, results)
        research_result += "\n" + f">{key} : {question}" + "\n\n" + reformed_results + "\n" + "---"
    return research_result

In [8]:
# Collect insights for the 20 questions
# research_result = search_insights(json_response)

In [9]:
# display(Markdown(research_result))

## Graph

### Graph State

In [10]:
class GraphState(TypedDict):
    product: str  
    region: str
    target_market: NotRequired[str]
    competitors: NotRequired[str]
    pricing_strategy: NotRequired[str]
    context: NotRequired[str]
    questions_generated: str
    questions_formatted: Dict[str, str]  # Corrected spelling
    research_result: str

### Nodes

In [11]:
def generate_questions(state):
    print("---QUESTIONNING---")
    product = state["product"]
    region = state["region"]
    target_market = state.get("target_market", "")
    competitors = state.get("competitors", "")
    pricing_strategy = state.get("pricing_strategy", "")
    context = state.get("context", "")

    questions_response = reflector_chain.invoke({
        "product": product, 
        "region": region, 
        "target_market": target_market, 
        "competitors": competitors, 
        "pricing_strategy": pricing_strategy, 
        "contextual_info": context
    })
    
    return {"questions_generated": questions_response}

In [12]:
def json_formatter(state):
    print("---FORMATTING---")
    questions_generated = state["questions_generated"]

    formatted_questions = json_transformer_chain.invoke({"questions_generated": questions_generated})
    
    # Check if formatted_questions is already a dictionary
    if isinstance(formatted_questions, dict):
        formatted_questions_dict = formatted_questions
    else:
        # If it's a string, parse it as JSON
        formatted_questions_dict = json.loads(formatted_questions)

    return {"questions_formatted": formatted_questions_dict}  # Note the correct spelling here

In [13]:
def research(state):
    print("---RESEARCHING---")
    questions = state["questions_formatted"]
    
    research = search_insights(questions)
    
    return {"research_result": research}

### Control Flow

In [15]:

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate_questions", generate_questions)
workflow.add_node("json_formatter", json_formatter)
workflow.add_node("research", research)

# Add edges
workflow.add_edge(START, "generate_questions")
workflow.add_edge("generate_questions", "json_formatter")
workflow.add_edge("json_formatter", "research")
workflow.add_edge("research", END)

# Set the entry point
workflow.set_entry_point("generate_questions")

# Compile
graph = workflow.compile()
# display(Image(graph.get_graph().draw_mermaid_png()))

In [17]:
inputs = {
    "product": "Nothing Phone 2A", 
    "region": "India",
    "target_market": "Tech-savvy consumers aged 18-35",
    "competitors": "Apple, Samsung, Xiaomi",
    "pricing_strategy": "Value-based pricing",
    "context": "The product is a new smartphone model with advanced camera features and long battery life."
}

for event in graph.stream(inputs, stream_mode="values"):
    research = event
    print(event)

{'product': 'Nothing Phone 2A', 'region': 'India', 'target_market': 'Tech-savvy consumers aged 18-35', 'competitors': 'Apple, Samsung, Xiaomi', 'pricing_strategy': 'Value-based pricing', 'context': 'The product is a new smartphone model with advanced camera features and long battery life.'}
---QUESTIONNING---
{'product': 'Nothing Phone 2A', 'region': 'India', 'target_market': 'Tech-savvy consumers aged 18-35', 'competitors': 'Apple, Samsung, Xiaomi', 'pricing_strategy': 'Value-based pricing', 'context': 'The product is a new smartphone model with advanced camera features and long battery life.', 'questions_generated': "1. What are the current market trends for smartphones in India, and how does the Nothing Phone 2A fit into this landscape?\n2. What are the unique selling points (USPs) of the Nothing Phone 2A compared to its competitors, and how can these be leveraged to justify a premium price?\n3. What are the current prices of similar smartphones from competitors like Apple, Samsung,

In [18]:
display(Markdown(research["research_result"]))

---
>1 : What are the current market trends for smartphones in India, and how does the Nothing Phone 2A fit into this landscape?

**Indian Smartphone Market Overview**

*India shipped 69 million smartphones in H1 2024, reflecting a 7.2% YoY growth.*

**Market Performance by Quarter:**

In Q1 2024, India maintained a strong performance, reaching 35.3 million units, with channels embracing multiple new offerings due to an improved inventory situation. This resulted in a 15% YoY growth.

**Indian Smartphone Market Share Report (H1 2024)**
| Aspect | Value |
| --- | --- |
| Q1 2024 Units Shipped | 35.3 million |
| Q1 2024 YoY Growth | 15% |
| H1 2024 Units Shipped | 69 million |
| H1 2024 YoY Growth | 7.2% |

**Market Projections and Challenges**

* A 5% growth is projected in 2024, driven by 5G adoption and device upgrades.
* Market challenges include cost pressures and extended replacement cycles.

**2023 Market Performance**

* The Indian smartphone market shipped 148.6 million units in 2023, witnessing a 2% YoY decline.
* 5G smartphone shipments captured their highest-ever share of 71% in volume terms.

"Our research suggests that despite the decline in 2023, the Indian smartphone market is poised for growth in 2024, driven by 5G adoption and device upgrades."
---
>2 : What are the unique selling points (USPs) of the Nothing Phone 2A compared to its competitors, and how can these be leveraged to justify a premium price?

**Unique Selling Points (USPs) of the Nothing Phone 2A**

* *Innovative Features*: Introducing unique features to a broader audience, setting it apart from competitors.
* *Customization Options*: Offers masses of customization, allowing users to personalize their experience.
* *Design*: A fun, premium-looking design that challenges the notion that low-cost phones can't look good.

**Key Features and Specifications**

* _Price_: Starting at $350/£319, making it an attractive option for those on a budget.
* _Display_: 120 Hz AMOLED screen with thin bezels, providing an excellent visual experience.
* _Performance_: MediaTek Dimensity 7200 Pro SoC, suitable for everyday use.

**What Sets it Apart**

The Nothing Phone 2a's ability to balance price and features, making it a great value proposition. The phone's premium look and feel, despite its low cost, make it an attractive option for those who want a high-end experience without the hefty price tag.

**Leveraging USPs to Justify a Premium Price**

* Emphasize the unique features and customization options that set it apart from competitors.
* Highlight the phone's premium design and build quality, despite its affordable price.
* Focus on the excellent display and performance, demonstrating that the phone can compete with more expensive options.

By emphasizing these key points, the Nothing Phone 2a can be positioned as a premium product that offers excellent value for money, justifying a slightly higher price point than its competitors.
---

In [19]:
print(research["research_result"])

---
>1 : What are the current market trends for smartphones in India, and how does the Nothing Phone 2A fit into this landscape?

**Indian Smartphone Market Overview**

*India shipped 69 million smartphones in H1 2024, reflecting a 7.2% YoY growth.*

**Market Performance by Quarter:**

In Q1 2024, India maintained a strong performance, reaching 35.3 million units, with channels embracing multiple new offerings due to an improved inventory situation. This resulted in a 15% YoY growth.

**Indian Smartphone Market Share Report (H1 2024)**
| Aspect | Value |
| --- | --- |
| Q1 2024 Units Shipped | 35.3 million |
| Q1 2024 YoY Growth | 15% |
| H1 2024 Units Shipped | 69 million |
| H1 2024 YoY Growth | 7.2% |

**Market Projections and Challenges**

* A 5% growth is projected in 2024, driven by 5G adoption and device upgrades.
* Market challenges include cost pressures and extended replacement cycles.

**2023 Market Performance**

* The Indian smartphone market shipped 148.6 million units in